In [ ]:
# Step 1: Import necessary libraries
import numpy as np
import pandas as pd

# Step 2: Define the MLP class
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights randomly
        self.WH = np.random.rand(hidden_size, input_size) - 0.5  # Weights for input to hidden layer
        self.WO = np.random.rand(output_size, hidden_size) - 0.5  # Weights for hidden to output layer
        
        # Initialize outputs
        self.output_hidden = np.zeros(hidden_size)  # Hidden layer outputs
        self.output_neuron = np.zeros(output_size)  # Output layer outputs

    def sigmoid(self, x):
        """Compute the sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def feedforward(self, data_sample):
        """Perform the feedforward operation."""
        # Compute hidden layer outputs
        for i in range(len(self.output_hidden)):
            weighted_sum = np.dot(self.WH[i], data_sample)
            self.output_hidden[i] = self.sigmoid(weighted_sum)
        
        # Compute output layer values
        for i in range(len(self.output_neuron)):
            weighted_sum = np.dot(self.WO[i], self.output_hidden)
            self.output_neuron[i] = 1 if weighted_sum >= 0 else 0

    def test_error(self, target_map):
        """Check if there is an error in the output."""
        for i in range(len(self.output_neuron)):
            if target_map[i] != self.output_neuron[i]:
                return True
        return False

    def train(self, data_sample, target_map, learning_rate=0.01):
        """Update weights based on the error."""
        # Calculate output error
        output_error = target_map - self.output_neuron
        
        # Update weights between hidden and output layers
        for i in range(len(self.WO)):
            self.WO[i] += learning_rate * output_error[i] * self.output_hidden
        
        # Calculate hidden layer errors
        hidden_error = np.dot(self.WO.T, output_error) * self.output_hidden * (1 - self.output_hidden)
        
        # Update weights between input and hidden layers
        for i in range(len(self.WH)):
            self.WH[i] += learning_rate * hidden_error[i] * data_sample

# Step 3: Load dataset
def load_dataset(file_path):
    """Load the dataset and return input data and target labels."""
    dataset = pd.read_csv(file_path, header=None)
    data = dataset.iloc[:, :-1].values  # First 64 columns are input
    labels = dataset.iloc[:, -1].values  # Last column is the output
    return data, labels

# Step 4: Map target output
def create_target_map(target_output, output_size=10):
    """Create a one-hot encoded map for the target output."""
    target_map = np.zeros(output_size)
    target_map[target_output] = 1
    return target_map

# Step 5: Train and test the model
def train_and_test_mlp(data1, labels1, data2, labels2, hidden_size=15, iterations=10):
    """Train the MLP and test its performance."""
    input_size = data1.shape[1]
    output_size = 10
    mlp = MLP(input_size, hidden_size, output_size)

    for cycle in range(iterations):
        success = 0

        # Training phase
        for i in range(len(data1)):
            data_sample = data1[i]
            target_output = labels1[i]
            target_map = create_target_map(target_output)

            mlp.feedforward(data_sample)
            if mlp.test_error(target_map):
                mlp.train(data_sample, target_map)
            else:
                success += 1

        accuracy = success / len(data1)
        print(f"Cycle {cycle + 1}: Training Accuracy = {accuracy:.2%}")

    # Testing phase
    success = 0
    for i in range(len(data2)):
        data_sample = data2[i]
        target_output = labels2[i]
        target_map = create_target_map(target_output)

        mlp.feedforward(data_sample)
        if not mlp.test_error(target_map):
            success += 1

    test_accuracy = success / len(data2)
    print(f"Test Accuracy = {test_accuracy:.2%}")

# Step 6: Add placeholders for dataset paths
file1_path = "cw2DataSet1.csv"  # Placeholder for training data
file2_path = "cw2DataSet2.csv"  # Placeholder for testing data

# (Uncomment below when datasets are available on user's system)
# data1, labels1 = load_dataset(file1_path)
# data2, labels2 = load_dataset(file2_path)
# train_and_test_mlp(data1, labels1, data2, labels2)


